In [2]:
import pandas as pd
import numpy as np
import math
data_file = pd.read_csv('result3.csv')
data_file.describe()

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,year,quart,return,last_return
count,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,...,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000,26862.000000
mean,1.934676,1.909732,22.233223,2.926143,14.428193,21.988104,84.557912,51.578900,6.046190,28.381086,...,3.543460,4.363080,1.619221,1.491252,1.409777,0.915031,2013.203820,2.516454,1.035751,1.031672
std,1.786361,1.763966,16.763664,2.784431,12.565569,22.502162,66.569252,46.054232,5.533868,28.657611,...,4.993376,6.211652,1.014030,0.980491,1.035513,0.624531,4.069402,1.129035,0.162546,0.158276
min,-1.100000,-1.070000,-5.990000,-1.700000,-7.440000,-9.460000,-47.830000,-55.992500,-2.710000,-39.430000,...,-2.720000,-2.880000,-0.290000,0.000500,-0.280000,-0.060000,2006.000000,1.000000,0.127124,0.127124
25%,0.640000,0.630000,9.810000,0.860000,4.520000,5.500000,32.710000,9.972500,1.750000,1.260000,...,0.380000,0.440000,0.830000,0.790000,0.640000,0.480000,2010.000000,2.000000,0.952890,0.953180
50%,1.510000,1.480000,17.710000,2.140000,10.500000,12.980000,72.980000,42.955000,3.980000,21.605000,...,1.930000,2.100000,1.390000,1.190000,1.100000,0.750000,2013.000000,3.000000,1.039123,1.033885
75%,2.910000,2.870000,31.540000,4.400000,21.017500,31.747500,123.770000,78.367500,8.440000,50.600000,...,6.010000,7.097500,2.180000,2.030000,1.900000,1.240000,2017.000000,4.000000,1.120891,1.116083
max,5.710000,5.650000,59.200000,8.920000,42.052500,69.282500,225.350000,149.192500,17.360000,108.590000,...,14.170000,17.092500,3.760000,3.710000,3.500000,2.220000,2020.000000,4.000000,3.239215,2.890282


In [3]:
data_test = data_file[data_file['year']>=2017]
data_train = data_file[data_file['year']<2017]
all_input = data_file.drop('return',axis=1).drop('year',axis=1).drop('quart',axis=1)
all_output = data_file['return']
inputall = np.array(all_input)
outputall = np.array(all_output)
train_input = data_train.drop('return',axis=1).drop('year',axis=1).drop('quart',axis=1)
train_output = data_train['return']
inputtrain = np.array(train_input)
outputtrain = np.array(train_output)
test_input = data_test.drop('return',axis=1).drop('year',axis=1).drop('quart',axis=1)
test_output = data_test['return']
inputtest = np.array(test_input)
outputtest = np.array(test_output)

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [9]:
class VariationalDense:
    
    def __init__(self, n_in, n_out):
        
        self.W_mu = tf.Variable(tf.truncated_normal([n_in, n_out], stddev=stddev_var))
        self.W_logsigma = tf.Variable(tf.truncated_normal([n_in, n_out], mean = 0., stddev=stddev_var))
        self.b_mu = tf.Variable(tf.zeros([n_out]))
        self.b_logsigma = tf.Variable(tf.zeros([n_out]))
        
        self.epsilon_w = self.get_random([n_in,n_out],mu=0., std_dev=epsilon_prior)
        self.epsilon_b = self.get_random([n_out], mu=0., std_dev =epsilon_prior)
        
        self.W = self.W_mu + tf.multiply(tf.log(1. + tf.exp(self.W_logsigma)), self.epsilon_w)
        self.b = self.b_mu + tf.multiply(tf.log(1. + tf.exp(self.b_logsigma)), self.epsilon_b)
        
    def __call__(self, x, activation=tf.identity):
        output = activation(tf.matmul(x,self.W) + self.b)
        output = tf.squeeze(output)
        return output
    
    
    def log_gaussian(self, x, mu, sigma):
        return -0.5*tf.log(2*np.pi)-tf.log(sigma)-(x-mu)**2/(2*sigma**2)

    def get_random(self, shape, mu, std_dev):
        return tf.random_normal(shape, mean=mu, stddev=std_dev)
    
    def regularization(self):
        
        sample_log_pw, sample_log_qw= 0. , 0.
        
        sample_log_pw += tf.reduce_sum(self.log_gaussian(self.W, 0., sigma_prior))
        sample_log_pw += tf.reduce_sum(self.log_gaussian(self.b, 0., sigma_prior))
        
        sample_log_qw += tf.reduce_sum(self.log_gaussian(self.W, self.W_mu, tf.log(1. + tf.exp(self.W_logsigma))))
        sample_log_qw += tf.reduce_sum(self.log_gaussian(self.b, self.b_mu, tf.log(1. + tf.exp(self.b_logsigma))))
        
        regulizer = tf.reduce_sum((sample_log_qw-sample_log_pw))
        
        return regulizer

In [15]:
# Create the Model

sigma_prior = 0.1
epsilon_prior = 0.1
n_epochs = 10000
stddev_var = 0.001
n_sample = inputtrain.shape[0]
feature_size = inputtrain.shape[1]

lr = 0.0000005
r_lambda = 0
nlayers1 = 256
nlayers2 = 256

# Place holder of Network inputs and outputs
model_x = tf.placeholder(tf.float32, shape=[None, feature_size])
model_y = tf.placeholder(tf.float32, shape=[None])

#Define neural network
net_Layer_input = VariationalDense(feature_size, nlayers1)
net_Layer_hidden = VariationalDense(nlayers1, nlayers2)
net_Layer_output = VariationalDense(nlayers2, 1)

Input_Layer_output = net_Layer_input(model_x, tf.nn.softmax)
Hidden_Layer_output = net_Layer_hidden(Input_Layer_output, tf.nn.softmax)
net_pred =  net_Layer_output(Hidden_Layer_output)

# Define ELBO
sample_log_Likelihood = tf.reduce_sum(net_Layer_output.log_gaussian(model_y, net_pred, sigma_prior))
regularization_term = net_Layer_input.regularization() + net_Layer_output.regularization() + net_Layer_output.regularization()
elbo = - sample_log_Likelihood + regularization_term/nlayers1
train_step = tf.train.GradientDescentOptimizer(lr).minimize(elbo)

# Mean Square Error (Network Performance)
model_mse = tf.reduce_sum(tf.square(model_y - net_pred))/n_sample
model_mae = tf.reduce_sum(tf.abs(model_y - net_pred))/n_sample

mse = []
mae = []

# Train the MOdel
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        sess.run(train_step, feed_dict={model_x: inputtrain, model_y: outputtrain})
        mse.append(sess.run(model_mse, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        mae.append(sess.run(model_mae, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        print('iteration {}: mse: {:.4f}, mae: {:.4f}'.format(i,mse[i],mae[i]))
    n_post = 1000
    y_post = np.zeros((n_post, inputtest.shape[0]))
    for i in range(n_post):
        y_post[i] = sess.run(net_pred, feed_dict={model_x: inputtest})

iteration 0: mse: 0.0228, mae: 0.1974
iteration 1: mse: 0.0348, mae: 0.1090
iteration 2: mse: 0.0210, mae: 0.1668
iteration 3: mse: 0.0263, mae: 0.2253
iteration 4: mse: 0.0519, mae: 0.1184
iteration 5: mse: 0.0733, mae: 0.1715
iteration 6: mse: 0.0470, mae: 0.1054
iteration 7: mse: 0.0510, mae: 0.2258
iteration 8: mse: 0.0232, mae: 0.1526
iteration 9: mse: 0.0391, mae: 0.1384
iteration 10: mse: 0.0220, mae: 0.2157
iteration 11: mse: 0.0266, mae: 0.1162
iteration 12: mse: 0.0260, mae: 0.1129
iteration 13: mse: 0.0219, mae: 0.1121
iteration 14: mse: 0.0250, mae: 0.1750
iteration 15: mse: 0.0210, mae: 0.1298
iteration 16: mse: 0.0282, mae: 0.1180
iteration 17: mse: 0.0215, mae: 0.1157
iteration 18: mse: 0.0885, mae: 0.2907
iteration 19: mse: 0.0211, mae: 0.1092
iteration 20: mse: 0.0216, mae: 0.1312
iteration 21: mse: 0.0318, mae: 0.1081
iteration 22: mse: 0.0398, mae: 0.1132
iteration 23: mse: 0.0260, mae: 0.1749
iteration 24: mse: 0.0330, mae: 0.1174
iteration 25: mse: 0.0279, mae: 0.1

In [32]:
mmse =((y-outputtest)*(y-outputtest)).sum()/7137

In [33]:
mmse

0.04207147660149469

In [34]:
mmae = (abs(y-outputtest).sum())/7137

In [35]:
mmae

0.14533283775537795

In [42]:
mmse1

0.04140538225565025

In [43]:
mmae1

0.1436045077827589

In [45]:
# Create the Model

sigma_prior = 0.1
epsilon_prior = 0.1
n_epochs = 5000
stddev_var = 0.001
n_sample = inputtrain.shape[0]
feature_size = inputtrain.shape[1]

lr = 0.0000005
r_lambda = 0
nlayers1 = 32
nlayers2 = 64

# Place holder of Network inputs and outputs
model_x = tf.placeholder(tf.float32, shape=[None, feature_size])
model_y = tf.placeholder(tf.float32, shape=[None])

#Define neural network
net_Layer_input = VariationalDense(feature_size, nlayers1)
net_Layer_hidden = VariationalDense(nlayers1, nlayers2)
net_Layer_output = VariationalDense(nlayers2, 1)

Input_Layer_output = net_Layer_input(model_x, tf.nn.softmax)
Hidden_Layer_output = net_Layer_hidden(Input_Layer_output, tf.nn.softmax)
net_pred =  net_Layer_output(Hidden_Layer_output)

# Define ELBO
sample_log_Likelihood = tf.reduce_sum(net_Layer_output.log_gaussian(model_y, net_pred, sigma_prior))
elbo = - sample_log_Likelihood
train_step = tf.train.GradientDescentOptimizer(lr).minimize(elbo)

# Mean Square Error (Network Performance)
model_mse = tf.reduce_sum(tf.square(model_y - net_pred))/n_sample
model_mae = tf.reduce_sum(tf.abs(model_y - net_pred))/n_sample

mse2 = []
mae2 = []

# Train the MOdel
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        sess.run(train_step, feed_dict={model_x: inputtrain, model_y: outputtrain})
        mse2.append(sess.run(model_mse, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        mae2.append(sess.run(model_mae, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        print('iteration {}: mse: {:.4f}, mae: {:.4f}'.format(i,mse2[i],mae2[i]))
    n_post = 1000
    y_post2 = np.zeros((n_post, inputtest.shape[0]))
    for i in range(n_post):
        y_post2[i] = sess.run(net_pred, feed_dict={model_x: inputtest})

iteration 0: mse: 0.0226, mae: 0.1350
iteration 1: mse: 0.0212, mae: 0.1121
iteration 2: mse: 0.0219, mae: 0.1135
iteration 3: mse: 0.0220, mae: 0.1771
iteration 4: mse: 0.0274, mae: 0.1136
iteration 5: mse: 0.0211, mae: 0.1421
iteration 6: mse: 0.1154, mae: 0.1827
iteration 7: mse: 0.0210, mae: 0.1087
iteration 8: mse: 0.0262, mae: 0.1047
iteration 9: mse: 0.0211, mae: 0.1140
iteration 10: mse: 0.0224, mae: 0.1894
iteration 11: mse: 0.0238, mae: 0.1396
iteration 12: mse: 0.0240, mae: 0.1051
iteration 13: mse: 0.0215, mae: 0.1248
iteration 14: mse: 0.0293, mae: 0.1047
iteration 15: mse: 0.0859, mae: 0.2694
iteration 16: mse: 0.0241, mae: 0.1226
iteration 17: mse: 0.0255, mae: 0.1085
iteration 18: mse: 0.0323, mae: 0.1056
iteration 19: mse: 0.0253, mae: 0.1638
iteration 20: mse: 0.0249, mae: 0.1048
iteration 21: mse: 0.0604, mae: 0.1423
iteration 22: mse: 0.0237, mae: 0.1193
iteration 23: mse: 0.0513, mae: 0.1695
iteration 24: mse: 0.0230, mae: 0.1084
iteration 25: mse: 0.0279, mae: 0.1

In [46]:
# Create the Model

sigma_prior = 0.1
epsilon_prior = 0.1
n_epochs = 5000
stddev_var = 0.001
n_sample = inputtrain.shape[0]
feature_size = inputtrain.shape[1]

lr = 0.0000005
r_lambda = 0
nlayers1 = 64
nlayers2 = 128

# Place holder of Network inputs and outputs
model_x = tf.placeholder(tf.float32, shape=[None, feature_size])
model_y = tf.placeholder(tf.float32, shape=[None])

#Define neural network
net_Layer_input = VariationalDense(feature_size, nlayers1)
net_Layer_hidden = VariationalDense(nlayers1, nlayers2)
net_Layer_output = VariationalDense(nlayers2, 1)

Input_Layer_output = net_Layer_input(model_x, tf.nn.softmax)
Hidden_Layer_output = net_Layer_hidden(Input_Layer_output, tf.nn.softmax)
net_pred =  net_Layer_output(Hidden_Layer_output)

# Define ELBO
sample_log_Likelihood = tf.reduce_sum(net_Layer_output.log_gaussian(model_y, net_pred, sigma_prior))
elbo = - sample_log_Likelihood
train_step = tf.train.GradientDescentOptimizer(lr).minimize(elbo)

# Mean Square Error (Network Performance)
model_mse = tf.reduce_sum(tf.square(model_y - net_pred))/n_sample
model_mae = tf.reduce_sum(tf.abs(model_y - net_pred))/n_sample

mse3 = []
mae3 = []

# Train the MOdel
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        sess.run(train_step, feed_dict={model_x: inputtrain, model_y: outputtrain})
        mse3.append(sess.run(model_mse, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        mae3.append(sess.run(model_mae, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        print('iteration {}: mse: {:.4f}, mae: {:.4f}'.format(i,mse3[i],mae3[i]))
    n_post = 1000
    y_post3 = np.zeros((n_post, inputtest.shape[0]))
    for i in range(n_post):
        y_post3[i] = sess.run(net_pred, feed_dict={model_x: inputtest})

iteration 0: mse: 0.0337, mae: 0.1603
iteration 1: mse: 0.0220, mae: 0.1052
iteration 2: mse: 0.0418, mae: 0.1143
iteration 3: mse: 0.0217, mae: 0.1046
iteration 4: mse: 0.0212, mae: 0.1664
iteration 5: mse: 0.0247, mae: 0.1054
iteration 6: mse: 0.0212, mae: 0.1100
iteration 7: mse: 0.0339, mae: 0.1524
iteration 8: mse: 0.0213, mae: 0.1047
iteration 9: mse: 0.0210, mae: 0.1053
iteration 10: mse: 0.0414, mae: 0.1718
iteration 11: mse: 0.0226, mae: 0.1126
iteration 12: mse: 0.0215, mae: 0.1073
iteration 13: mse: 0.0269, mae: 0.1121
iteration 14: mse: 0.0357, mae: 0.1104
iteration 15: mse: 0.0246, mae: 0.1116
iteration 16: mse: 0.0210, mae: 0.1057
iteration 17: mse: 0.0222, mae: 0.1349
iteration 18: mse: 0.0219, mae: 0.1128
iteration 19: mse: 0.0335, mae: 0.1279
iteration 20: mse: 0.0222, mae: 0.1273
iteration 21: mse: 0.0235, mae: 0.1448
iteration 22: mse: 0.0210, mae: 0.1128
iteration 23: mse: 0.0218, mae: 0.1274
iteration 24: mse: 0.0252, mae: 0.1188
iteration 25: mse: 0.0222, mae: 0.1

In [47]:
# Create the Model

sigma_prior = 0.1
epsilon_prior = 0.1
n_epochs = 5000
stddev_var = 0.001
n_sample = inputtrain.shape[0]
feature_size = inputtrain.shape[1]

lr = 0.0000005
r_lambda = 0
nlayers1 = 128
nlayers2 = 256

# Place holder of Network inputs and outputs
model_x = tf.placeholder(tf.float32, shape=[None, feature_size])
model_y = tf.placeholder(tf.float32, shape=[None])

#Define neural network
net_Layer_input = VariationalDense(feature_size, nlayers1)
net_Layer_hidden = VariationalDense(nlayers1, nlayers2)
net_Layer_output = VariationalDense(nlayers2, 1)

Input_Layer_output = net_Layer_input(model_x, tf.nn.softmax)
Hidden_Layer_output = net_Layer_hidden(Input_Layer_output, tf.nn.softmax)
net_pred =  net_Layer_output(Hidden_Layer_output)

# Define ELBO
sample_log_Likelihood = tf.reduce_sum(net_Layer_output.log_gaussian(model_y, net_pred, sigma_prior))
elbo = - sample_log_Likelihood
train_step = tf.train.GradientDescentOptimizer(lr).minimize(elbo)

# Mean Square Error (Network Performance)
model_mse = tf.reduce_sum(tf.square(model_y - net_pred))/n_sample
model_mae = tf.reduce_sum(tf.abs(model_y - net_pred))/n_sample

mse4 = []
mae4 = []

# Train the MOdel
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        sess.run(train_step, feed_dict={model_x: inputtrain, model_y: outputtrain})
        mse4.append(sess.run(model_mse, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        mae4.append(sess.run(model_mae, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        print('iteration {}: mse: {:.4f}, mae: {:.4f}'.format(i,mse4[i],mae4[i]))
    n_post = 1000
    y_post4 = np.zeros((n_post, inputtest.shape[0]))
    for i in range(n_post):
        y_post4[i] = sess.run(net_pred, feed_dict={model_x: inputtest})

iteration 0: mse: 0.0370, mae: 0.1058
iteration 1: mse: 0.0217, mae: 0.1048
iteration 2: mse: 0.0246, mae: 0.1092
iteration 3: mse: 0.0244, mae: 0.1088
iteration 4: mse: 0.0299, mae: 0.1060
iteration 5: mse: 0.0320, mae: 0.1375
iteration 6: mse: 0.0273, mae: 0.1156
iteration 7: mse: 0.0210, mae: 0.1568
iteration 8: mse: 0.0431, mae: 0.1097
iteration 9: mse: 0.0291, mae: 0.1314
iteration 10: mse: 0.0472, mae: 0.1081
iteration 11: mse: 0.0235, mae: 0.1132
iteration 12: mse: 0.0261, mae: 0.1478
iteration 13: mse: 0.0295, mae: 0.1684
iteration 14: mse: 0.0216, mae: 0.1046
iteration 15: mse: 0.0213, mae: 0.1213
iteration 16: mse: 0.0213, mae: 0.1085
iteration 17: mse: 0.0665, mae: 0.2387
iteration 18: mse: 0.0231, mae: 0.1057
iteration 19: mse: 0.0311, mae: 0.1050
iteration 20: mse: 0.0213, mae: 0.1487
iteration 21: mse: 0.0381, mae: 0.2583
iteration 22: mse: 0.0222, mae: 0.1266
iteration 23: mse: 0.0235, mae: 0.1678
iteration 24: mse: 0.0219, mae: 0.1057
iteration 25: mse: 0.0238, mae: 0.1

In [48]:
y2 = y_post2.sum(axis=0)/1000
mmse2 =((y2-outputtest)*(y2-outputtest)).sum()/7137
mmae2 = (abs(y2-outputtest).sum())/7137

In [50]:
mmse2

0.041829489947899104

In [51]:
mmae2

0.1447016673942986

In [52]:
y3 = y_post3.sum(axis=0)/1000
mmse3 =((y3-outputtest)*(y3-outputtest)).sum()/7137
mmae3 = (abs(y3-outputtest).sum())/7137

In [53]:
mmse3

0.041061535350478455

In [55]:
mmae3

0.14282202156286847

In [61]:
y4 = y_post4.sum(axis=0)/1000
mmse4 =((y4-outputtest)*(y4-outputtest)).sum()/7137
mmae4 = (abs(y4-outputtest).sum())/7137

In [62]:
mmse4

0.041091957995848444

In [63]:
mmae4

0.1428700063676299

In [44]:
# Create the Model

sigma_prior = 0.1
epsilon_prior = 0.1
n_epochs = 5000
stddev_var = 0.001
n_sample = inputtrain.shape[0]
feature_size = inputtrain.shape[1]

lr = 0.0000005
r_lambda = 0
nlayers1 = 128
nlayers2 = 256

# Place holder of Network inputs and outputs
model_x = tf.placeholder(tf.float32, shape=[None, feature_size])
model_y = tf.placeholder(tf.float32, shape=[None])

#Define neural network
net_Layer_input = VariationalDense(feature_size, nlayers1)
net_Layer_hidden = VariationalDense(nlayers1, nlayers2)
net_Layer_output = VariationalDense(nlayers2, 1)

Input_Layer_output = net_Layer_input(model_x, tf.nn.softmax)
Hidden_Layer_output = net_Layer_hidden(Input_Layer_output, tf.nn.softmax)
net_pred =  net_Layer_output(Hidden_Layer_output)

# Define ELBO
sample_log_Likelihood = tf.reduce_sum(net_Layer_output.log_gaussian(model_y, net_pred, sigma_prior))
elbo = - sample_log_Likelihood
train_step = tf.train.GradientDescentOptimizer(lr).minimize(elbo)

# Mean Square Error (Network Performance)
model_mse = tf.reduce_sum(tf.square(model_y - net_pred))/n_sample
model_mae = tf.reduce_sum(tf.abs(model_y - net_pred))/n_sample

mse5 = []
mae5 = []

# Train the MOdel
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        sess.run(train_step, feed_dict={model_x: inputtrain, model_y: outputtrain})
        mse5.append(sess.run(model_mse, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        mae5.append(sess.run(model_mae, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        print('iteration {}: mse: {:.4f}, mae: {:.4f}'.format(i,mse5[i],mae5[i]))
    n_post = 1000
    y_post5 = np.zeros((n_post, inputtest.shape[0]))
    for i in range(n_post):
        y_post5[i] = sess.run(net_pred, feed_dict={model_x: inputtest})
    y_pred = np.zeros((n_post, pred_data.shape[0]))
    for i in range(n_post):
        y_pred[i] = sess.run(net_pred, feed_dict={model_x: pred_data})

iteration 0: mse: 0.0253, mae: 0.1228
iteration 1: mse: 0.0212, mae: 0.1259
iteration 2: mse: 0.0735, mae: 0.1255
iteration 3: mse: 0.0212, mae: 0.1159
iteration 4: mse: 0.0445, mae: 0.1234
iteration 5: mse: 0.0346, mae: 0.1207
iteration 6: mse: 0.0572, mae: 0.1064
iteration 7: mse: 0.0215, mae: 0.1688
iteration 8: mse: 0.0266, mae: 0.1976
iteration 9: mse: 0.0407, mae: 0.1473
iteration 10: mse: 0.0250, mae: 0.1187
iteration 11: mse: 0.0211, mae: 0.1435
iteration 12: mse: 0.0328, mae: 0.1054
iteration 13: mse: 0.0386, mae: 0.2105
iteration 14: mse: 0.0270, mae: 0.1296
iteration 15: mse: 0.0260, mae: 0.1154
iteration 16: mse: 0.0210, mae: 0.1472
iteration 17: mse: 0.0260, mae: 0.1504
iteration 18: mse: 0.0255, mae: 0.1805
iteration 19: mse: 0.0229, mae: 0.1116
iteration 20: mse: 0.0266, mae: 0.1356
iteration 21: mse: 0.0213, mae: 0.1461
iteration 22: mse: 0.0221, mae: 0.1996
iteration 23: mse: 0.0276, mae: 0.1068
iteration 24: mse: 0.0224, mae: 0.1355
iteration 25: mse: 0.0909, mae: 0.2

In [56]:
y5 = y_post5.sum(axis=0)/1000
mmse5 =((y5-outputtest)*(y5-outputtest)).sum()/7137
mmae5 = (abs(y5-outputtest).sum())/7137

In [57]:
mmse5

0.042066146133939435

In [58]:
mmae5

0.14531900427669692

In [8]:
data_pre = pd.read_csv('pred201912_withreturn.csv')
data_pre = data_pre.drop(['stock'],axis=1)
pred_data = np.array(data_pre)

In [16]:
data_pre = pd.read_csv('pred201909_withreturn.csv')
pred_data = np.array(data_pre)

In [21]:
data_pre = pd.read_csv('pred201906_withreturn.csv')
pred_data = np.array(data_pre)

In [25]:
data_pre = pd.read_csv('pred201903_withreturn.csv')
pred_data = np.array(data_pre)

In [26]:
# Create the Model

sigma_prior = 0.3
epsilon_prior = 0
n_epochs = 2000
stddev_var = 0.2
n_sample = inputtrain.shape[0]
feature_size = inputtrain.shape[1]

lr = 0.0000005
r_lambda = 0
nlayers1 = 128
nlayers2 = 256

# Place holder of Network inputs and outputs
model_x = tf.placeholder(tf.float32, shape=[None, feature_size])
model_y = tf.placeholder(tf.float32, shape=[None])

#Define neural network
net_Layer_input = VariationalDense(feature_size, nlayers1)
net_Layer_hidden = VariationalDense(nlayers1, nlayers2)
net_Layer_output = VariationalDense(nlayers2, 1)

Input_Layer_output = net_Layer_input(model_x, tf.nn.sigmoid)
Hidden_Layer_output = net_Layer_hidden(Input_Layer_output, tf.nn.sigmoid)
net_pred =  net_Layer_output(Hidden_Layer_output)

# Define ELBO
sample_log_Likelihood = tf.reduce_sum(net_Layer_output.log_gaussian(model_y, net_pred, sigma_prior))
elbo = - sample_log_Likelihood
train_step = tf.train.GradientDescentOptimizer(lr).minimize(elbo)

# Mean Square Error (Network Performance)
model_mse = tf.reduce_sum(tf.square(model_y - net_pred))/n_sample
model_mae = tf.reduce_sum(tf.abs(model_y - net_pred))/n_sample

mse5 = []
mae5 = []

# Train the MOdel
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        sess.run(train_step, feed_dict={model_x: inputtrain, model_y: outputtrain})
        mse5.append(sess.run(model_mse, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        mae5.append(sess.run(model_mae, feed_dict={model_x: inputtrain, model_y: outputtrain}))
        print('iteration {}: mse: {:.4f}, mae: {:.4f}'.format(i,mse5[i],mae5[i]))
    n_post = 1000
    y_post5 = np.zeros((n_post, inputtest.shape[0]))
    for i in range(n_post):
        y_post5[i] = sess.run(net_pred, feed_dict={model_x: inputtest})
    y_pred = np.zeros((n_post, pred_data.shape[0]))
    for i in range(1000):
        y_pred[i] = sess.run(net_pred, feed_dict={model_x: pred_data})

iteration 0: mse: 40.6408, mae: 6.3682
iteration 1: mse: 2675.1931, mae: 51.7190
iteration 2: mse: 176.3777, mae: 13.2636
iteration 3: mse: 7.2793, mae: 2.6933
iteration 4: mse: 5.7345, mae: 2.3894
iteration 5: mse: 4.5209, mae: 2.1203
iteration 6: mse: 3.5668, mae: 1.8819
iteration 7: mse: 2.8159, mae: 1.6706
iteration 8: mse: 2.2245, mae: 1.4831
iteration 9: mse: 1.7585, mae: 1.3167
iteration 10: mse: 1.3911, mae: 1.1689
iteration 11: mse: 1.1015, mae: 1.0377
iteration 12: mse: 0.8732, mae: 0.9212
iteration 13: mse: 0.6932, mae: 0.8178
iteration 14: mse: 0.5514, mae: 0.7260
iteration 15: mse: 0.4397, mae: 0.6446
iteration 16: mse: 0.3517, mae: 0.5726
iteration 17: mse: 0.2823, mae: 0.5088
iteration 18: mse: 0.2277, mae: 0.4524
iteration 19: mse: 0.1847, mae: 0.4026
iteration 20: mse: 0.1507, mae: 0.3587
iteration 21: mse: 0.1240, mae: 0.3201
iteration 22: mse: 0.1029, mae: 0.2864
iteration 23: mse: 0.0863, mae: 0.2571
iteration 24: mse: 0.0732, mae: 0.2317
iteration 25: mse: 0.0629, 

In [23]:
y_pred[0]

array([ 1.03942454,  1.04295647,  1.03804779,  0.92147636,  1.04283988,
        1.041875  ,  1.03222525,  1.0553726 ,  1.0365845 ,  1.03936386,
        1.05918622,  1.03603542,  0.90050441,  1.03804851,  1.04313338,
        1.04332387,  1.04030263,  1.03797197,  1.04281378,  1.02875841,
        1.03159213,  1.065588  ,  1.0535568 ,  1.01668787,  1.01366043,
        1.03053725,  1.04138124,  1.03549039,  1.04023814,  1.03889489,
        1.03404212,  1.04321933,  1.04278862,  1.0450294 ,  1.05621862,
        1.03247774,  1.04843032,  1.04607594,  1.05386555,  1.04773128,
        1.05534184,  1.04281366,  1.04287028,  1.03289855,  1.03253543,
        1.03412998,  1.04239619,  1.03340054,  1.03433239,  1.02837276,
        1.0450542 ,  1.03575361,  1.03805041,  1.05356205,  1.0301882 ,
        1.04281354,  1.04343379,  1.03803706,  1.03223133,  1.03864813,
        1.07305813,  1.04192543,  1.04305482,  1.028736  ,  1.03572881,
        1.03657305,  1.04645562,  1.03838634,  1.04012072,  1.04

In [20]:
pd.DataFrame(y_pred[0]).to_csv('bayes_result.csv')